In [2]:
import pandas as pd
import numpy as np

In [3]:
pay = pd.read_csv('../data/ceo data pay/ceo_data_pay_merged_r3000.csv')

In [4]:
pay = pay.drop(columns=['Unnamed: 0','pay_ratio'])

In [5]:
pay['median_worker_pay'] = pay['median_worker_pay'].str.replace(',','').str.replace('$','')
pay['salary'] = pay['salary'].str.replace(',','').str.replace('$','')
pay['industry'] = pay['industry'].str.replace('%20',' ')
pay['company_name'] = pay['company_name'].str.replace(',','')

C:\Users\jorge\AppData\Local\Temp\ipykernel_11976\2338073337.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pay['median_worker_pay'] = pay['median_worker_pay'].str.replace(',','').str.replace('$','')
C:\Users\jorge\AppData\Local\Temp\ipykernel_11976\2338073337.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pay['salary'] = pay['salary'].str.replace(',','').str.replace('$','')


In [6]:
pay['salary'] = pay['salary'].astype(int)
pay['median_worker_pay'] = pay['median_worker_pay'].astype(int)

In [7]:

pay.drop(index=pay[(pay['salary'] == 0) | (pay['median_worker_pay'] == 0)].index,inplace=True)

In [8]:
pay['ratio'] = round(pay['salary'] / pay['median_worker_pay'])
pay['ratio'] = pay['ratio'].astype(int)

In [9]:
pay.to_csv('../data/ceo data pay/clean.csv',index=False)

In [10]:
lays = pd.read_csv('../data/tech layoffs/tech_layoffs.csv')

In [11]:
lays = lays.drop(columns=['additional_notes'])
lays = lays.drop(index=lays[lays['company'].duplicated()].index)
lays = lays.reset_index(drop=True)

In [12]:
def new_col(col,*args):
    z = lays['industry'].apply(lambda x : x.lower())
    lays[col] = z.str.contains(col.lower())
    for arg in args:
        lays[col]+= z.str.contains(arg.lower())

In [13]:
new_col('Fintech','finance','financial','payment')
new_col('Real estate','proptech')
new_col('SaaS','software','crm')
new_col('Ecommerce','e-commerce','sales','retail')
new_col('Logistics','shipping','transport')
new_col('Energy','solar','renewable')
new_col('Marketing','Advertising','advert','adtech')
new_col('Food','drink','beverage')
new_col('Health','wellness','beauty')
new_col('Crypto','blockchain','nft')
new_col('Journalism','news','media','sport')
new_col('Analytics')
new_col('Artificial Intelligence','ai','machine','predictive')
new_col('Security','surveillance')
new_col('Cloud')
new_col('IT','information technology','productivity','HR','human','customer service')
new_col('Insurance','insur')
new_col('Video Games','game','gaming')

In [14]:
company_industry = lays.copy()
company_industry = company_industry.drop(columns = ['total_layoffs','impacted_workforce_percentage','reported_date',
                          'industry','headquarter_location','sources','status'])

In [15]:
company_industry['Industries'] = ' '
for l,i in enumerate(company_industry.itertuples()):
    industry_list = []
    for k,j in enumerate(i[2:-1]):
        if j:
            industry_list.append(k+1)
    company_industry.at[ l , 'Industries'] = industry_list

In [16]:
industry = pd.DataFrame(columns = ['id','industry'])
for i,j in enumerate(company_industry.columns[1:-1]):
    industry.at[i,'industry'] = j
    industry.at[i,'id'] = i+1
company_industry = company_industry[['company','Industries']]

In [17]:
company_industry = company_industry.explode('Industries')
company_industry =  company_industry.drop(index = company_industry[company_industry['Industries'].isna()].index)
company_industry.reset_index(drop=True)
lays = lays[lays.columns[:8]]

In [18]:
lays['impacted_workforce_percentage'][lays['impacted_workforce_percentage']=='Unclear'] = np.nan
lays['total_layoffs'][lays['total_layoffs']=='Unclear'] = np.nan

In [19]:
lays['total_layoffs'] = lays['total_layoffs'].astype(float)
lays['impacted_workforce_percentage'] = lays['impacted_workforce_percentage'].astype(float)

In [20]:
lays.to_csv('../data/tech layoffs/layoffs.csv',index=False)
industry.to_csv('../data/tech layoffs/industry.csv',index=False)
company_industry.to_csv('../data/tech layoffs/company_industry.csv',index=False)


### Fuzzy Merge Layoffs and CEO Pays

In [24]:
import difflib

def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.6):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

In [56]:
y = lays.copy()
y['company'] = y['company'].str.lower()
y['company'] = y['company'].str.replace('inc.','',regex=False)
y['company'] = y['company'].str.replace('inc','',regex=False)
y['company'] = y['company'].str.replace('therapeutics','',regex=False)
y['company'] = y['company'].str.replace('technologies','',regex=False)
y['company'] = y['company'].str.replace('financial','',regex=False)
y['company'] = y['company'].str.replace('systems','',regex=False)
y['company'] = y['company'].str.replace('commerce','',regex=False)
y['company'] = y['company'].str.replace('security','',regex=False)
y['company'] = y['company'].str.replace('health','',regex=False)
y['company'] = y['company'].str.replace('.com','',regex=False)
y['company'] = y['company'].str.strip()
z = pay.copy()
z['company_name'] = z['company_name'].str.lower()
z['company_name'] = z['company_name'].str.replace('inc.','',regex=False)
z['company_name'] = z['company_name'].str.replace('inc','',regex=False)
z['company_name'] = z['company_name'].str.replace('therapeutics','',regex=False)
z['company_name'] = z['company_name'].str.replace('technologies','',regex=False)
z['company_name'] = z['company_name'].str.replace('financial','',regex=False)
z['company_name'] = z['company_name'].str.replace('systems','',regex=False)
z['company_name'] = z['company_name'].str.replace('commerce','',regex=False)
z['company_name'] = z['company_name'].str.replace('security','',regex=False)
z['company_name'] = z['company_name'].str.replace('health','',regex=False)
z['company_name'] = z['company_name'].str.replace('.com','',regex=False)
z['company_name'] = z['company_name'].str.strip()

In [59]:
x = fuzzy_merge(y,z, left_on='company',right_on='company_name',cutoff=0.9)

In [62]:
x.to_csv('../data/tech layoffs/merged.csv',index=False)